In [1]:
# import psycopg2
import pandas as pd
import ipywidgets
import os
import sqlite3
import numpy as np
import plotly.graph_objects as go
from docx import Document
# !pip install python-docx
# from docx import Document
from datetime import datetime as dt
import random

from ipywidgets import widgets, interactive_output, interact, interactive, fixed, widget 

In [15]:
df = pd.read_excel("invoice.xlsx",).ffill().set_index("Cliente")
clients = pd.read_excel("inventario.xlsm", sheet_name="clientes")
inventario = pd.read_excel("inventario.xlsm", sheet_name="inventario")

In [139]:
client = df.index.values[0]

address = clients.loc[clients["tienda"] == client, "direccion"].iloc[0]

email = clients.loc[clients["tienda"] == client, "email"].iloc[0]

In [108]:
NIT = "123456789-0"
invoice = Document()

invoice.add_heading("Factura", 0)

p_1 = invoice.add_paragraph()
p_1.add_run("NIT: ").bold = True
p_1.add_run(NIT)
p_2 = invoice.add_paragraph()
p_2.add_run("Direccion: ").bold = True
p_2.add_run(address)
p_3 = invoice.add_paragraph()
p_3.add_run("email: ").bold = True
p_3.add_run(email)
invoice.add_paragraph("")
p_4 = invoice.add_paragraph()
p_4.add_run("Fecha: ").bold = True
p_4.add_run(dt.today().strftime("%d/%m/%Y"))
p_5 = invoice.add_paragraph()
p_5.add_run("No de Factura: ").bold = True
p_5.add_run(f"{random.randint(0, 1000000)}")
invoice.add_paragraph("")
invoice.add_heading("Detalle de la Compra", level=1)
invoice.add_paragraph("")

table = invoice.add_table(rows=1, cols=4)

for i,j in zip(table.rows[0].cells, df.columns):
    i.text = j
    
for i in df.values:
    new_row = table.add_row().cells
    for j,k in zip(new_row, i):
        j.text = f"{k}"

last_row = table.add_row().cells

last_row[0].text = "Total"
last_row[-1].text = f"{df['Valor'].sum()}"

invoice.save("invoice.docx")



In [11]:
inventario[inventario["producto"].isin(df["Producto"])]

,producto,cantidad en bodega,cantidad en tienda,precio
1,"Gin - Gilbeys London, Dry",91,42,37.06
2,"Dill - Primerba, Paste",153,33,58.03
3,Juice - Happy Planet,155,97,29.25
6,Pheasants - Whole,21,86,40.35
10,"Gin - Gilbeys London, Dry",157,31,62.96
43,Juice - Happy Planet,16,17,16.21


In [17]:
for i, j in df[["Producto", "Cantidad"]].values:

    inventario.loc[inventario["producto"] == i, "cantidad en bodega"] -= j
    
    if inventario.loc[inventario["producto"] == i, "cantidad en bodega"].iloc[0] < 0:
        
        inventario.loc[inventario["producto"] == i, "cantidad en tienda"] += inventario.loc[inventario["producto"] == i, "cantidad en bodega"]
        inventario.loc[inventario["producto"] == i, "cantidad en bodega"] = 0

with pd.ExcelWriter("inventario.xlsm", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    inventario.to_excel(writer, sheet_name="inventario", index=False)

# df[["Producto", "Cantidad"]].values



PermissionError: [Errno 13] Permission denied: 'inventario.xlsm'

In [13]:
inventario[inventario["producto"].isin(df["Producto"])]

,producto,cantidad en bodega,cantidad en tienda,precio
1,"Gin - Gilbeys London, Dry",86,42,37.06
2,"Dill - Primerba, Paste",149,33,58.03
3,Juice - Happy Planet,152,97,29.25
6,Pheasants - Whole,14,86,40.35
10,"Gin - Gilbeys London, Dry",152,31,62.96
43,Juice - Happy Planet,13,17,16.21


In [14]:
df

,Producto,Cantidad,Valor Und,Valor
Cliente,,,,
Nlounge,Pheasants - Whole,7,40.35,282.45
Nlounge,Juice - Happy Planet,3,29.25,87.75
Nlounge,"Dill - Primerba, Paste",4,58.03,232.12
Nlounge,"Gin - Gilbeys London, Dry",5,37.06,185.30
